<a href="https://colab.research.google.com/github/KevinRocha1/MLHW/blob/main/MLHW.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#!/usr/bin/env python3
"""
assignment_solution.py

Complete pipeline for:
- Data preprocessing
- K-means clustering (from scratch, KMeans++)
- Elbow method + silhouette
- Cluster analysis & visualizations
- Regression (Linear + Polynomial degree 2)
- Saves figures to ./results and REPORT.pdf

Usage:
    python assignment_solution.py
"""

import os
import textwrap
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, mean_absolute_error, silhouette_score

# -------------------------
# Configuration
# -------------------------
DATA_PATH = "product_sales.csv"
RESULTS_DIR = "results"
REPORT_PATH = "REPORT.pdf"
RANDOM_STATE = 42
K_RANGE = list(range(2, 9))  # 2..8 inclusive

# Create results folder
os.makedirs(RESULTS_DIR, exist_ok=True)

# -------------------------
# Utility functions
# -------------------------
def cap_outliers(series):
    """Cap values using IQR winsorization (1.5*IQR)"""
    q1 = series.quantile(0.25)
    q3 = series.quantile(0.75)
    iqr = q3 - q1
    lower = q1 - 1.5 * iqr
    upper = q3 + 1.5 * iqr
    return series.clip(lower=lower, upper=upper)

# K-means from scratch with KMeans++ initialization
def kmeans_custom(X, k, max_iters=300, tol=1e-4, init='kmeans++', random_state=None):
    """
    X: numpy array shape (n_samples, n_features)
    returns: labels (n_samples,), centroids (k, n_features), wcss (float)
    """
    if random_state is not None:
        np.random.seed(random_state)

    n_samples, n_features = X.shape

    # KMeans++ init
    if init == 'kmeans++':
        centroids = np.zeros((k, n_features))
        # pick first centroid randomly
        idx = np.random.choice(n_samples)
        centroids[0] = X[idx]
        for ci in range(1, k):
            # compute squared distances to nearest existing centroid
            dists_sq = np.min(np.linalg.norm(X[:, None, :] - centroids[:ci][None, :, :], axis=2)**2, axis=1)
            probs = dists_sq / dists_sq.sum()
            # choose next centroid index by weighted probability
            next_idx = np.random.choice(n_samples, p=probs)
            centroids[ci] = X[next_idx]
    else:
        # random unique samples
        indices = np.random.choice(n_samples, k, replace=False)
        centroids = X[indices].astype(float)

    for it in range(max_iters):
        # assign labels
        dists = np.linalg.norm(X[:, None, :] - centroids[None, :, :], axis=2)  # (n_samples, k)
        labels = np.argmin(dists, axis=1)

        # update centroids
        new_centroids = np.array([
            X[labels == j].mean(axis=0) if np.any(labels == j) else centroids[j]
            for j in range(k)
        ])

        shift = np.linalg.norm(new_centroids - centroids)
        centroids = new_centroids
        if shift < tol:
            break

    # compute WCSS
    wcss = np.sum((X - centroids[labels])**2)
    return labels, centroids, wcss

# -------------------------
# Load dataset
# -------------------------
df = pd.read_csv(DATA_PATH)
print(f"Loaded {DATA_PATH} with shape {df.shape}")

# -------------------------
# Preprocessing
# -------------------------
# Columns expected by the assignment:
numeric_cols = ["price", "cost", "units_sold", "promotion_frequency", "shelf_level", "profit"]
cat_cols = ["category"]

# 1) Missing values
df_clean = df.copy()

# Impute numeric with median
for col in numeric_cols:
    if col in df_clean.columns:
        median_val = df_clean[col].median()
        df_clean[col] = df_clean[col].fillna(median_val)

# Impute categorical with mode
for col in cat_cols:
    if col in df_clean.columns:
        if df_clean[col].isnull().any():
            mode_val = df_clean[col].mode().iloc[0] if not df_clean[col].mode().empty else "Unknown"
            df_clean[col] = df_clean[col].fillna(mode_val)

# Fill product_name if missing
if "product_name" in df_clean.columns and "product_id" in df_clean.columns:
    df_clean["product_name"] = df_clean["product_name"].fillna(df_clean["product_id"].apply(lambda x: f"Product_{x}"))

# 2) Outlier capping (IQR)
for col in numeric_cols:
    if col in df_clean.columns:
        df_clean[col] = cap_outliers(df_clean[col])

# 3) Scaling for clustering: Z-score
X_k = df_clean[numeric_cols].copy()
scaler_k = StandardScaler()
X_k_scaled = scaler_k.fit_transform(X_k.values)

# -------------------------
# K-means: elbow + silhouette
# -------------------------
wcss_values = {}
silhouette_vals = {}
for k in K_RANGE:
    labels_k, centroids_k, wcss_k = kmeans_custom(X_k_scaled, k, init='kmeans++', random_state=RANDOM_STATE)
    wcss_values[k] = wcss_k
    try:
        sil = silhouette_score(X_k_scaled, labels_k)
    except Exception:
        sil = np.nan
    silhouette_vals[k] = sil

# Save elbow image
plt.figure()
plt.plot(list(wcss_values.keys()), list(wcss_values.values()), marker='o')
plt.title("Elbow Curve (k vs WCSS)")
plt.xlabel("k (number of clusters)")
plt.ylabel("WCSS")
plt.grid(True)
plt.savefig(os.path.join(RESULTS_DIR, "elbow_curve.png"))
plt.close()

# Choose best k by silhouette (fallback) - user can override later
valid_sil = {k: v for k, v in silhouette_vals.items() if not np.isnan(v)}
if valid_sil:
    best_k = max(valid_sil, key=valid_sil.get)
else:
    # fallback to elbow heuristic: k at which WCSS drop slows; for automation take k=3
    best_k = 3

print(f"Recommended k (by silhouette): {best_k}")

# Final clustering with chosen k
labels_final, centroids_final, wcss_final = kmeans_custom(X_k_scaled, best_k, init='kmeans++', random_state=RANDOM_STATE)
df_clean["cluster"] = labels_final

# Compute cluster statistics (on original scale)
cluster_stats = df_clean.groupby("cluster").agg(
    n_products=("product_id", "count"),
    avg_price=("price", "mean"),
    avg_units_sold=("units_sold", "mean"),
    avg_profit=("profit", "mean"),
    avg_promotion_frequency=("promotion_frequency", "mean")
).reset_index()

# Heuristic naming for clusters
cluster_names = {}
price_q1 = df_clean["price"].quantile(0.33)
price_q2 = df_clean["price"].quantile(0.66)
units_q1 = df_clean["units_sold"].quantile(0.33)
units_q2 = df_clean["units_sold"].quantile(0.66)

for _, r in cluster_stats.iterrows():
    cid = int(r["cluster"])
    p = r["avg_price"]
    u = r["avg_units_sold"]
    if p <= price_q1 and u >= units_q2:
        cluster_names[cid] = "Budget Best-Sellers"
    elif p >= price_q2 and u <= units_q1:
        cluster_names[cid] = "Premium Low-Volume"
    else:
        cluster_names[cid] = "Mid-Range Steady"

cluster_stats["cluster_name"] = cluster_stats["cluster"].map(cluster_names)

# Save cluster scatter (price vs units_sold)
plt.figure(figsize=(8,6))
for cid in sorted(df_clean["cluster"].unique()):
    subset = df_clean[df_clean["cluster"] == cid]
    plt.scatter(subset["price"], subset["units_sold"], label=f"Cluster {cid}: {cluster_names[cid]}", alpha=0.7)
# mark centroids mapped back to original scale (centroids_final correspond to scaled numeric_cols order)
centroids_orig = scaler_k.inverse_transform(centroids_final)
for idx, c in enumerate(centroids_orig):
    # price at index 0, units_sold index 2 (per numeric_cols)
    plt.scatter(c[0], c[2], marker="X", s=200, edgecolor="k")
plt.title("Clusters: Price vs Units Sold (centroids marked)")
plt.xlabel("Price ($)")
plt.ylabel("Units Sold (per month)")
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.savefig(os.path.join(RESULTS_DIR, "cluster_price_units.png"))
plt.close()

# -------------------------
# Regression (predict 'profit')
# -------------------------
# Prepare features: numeric + one-hot category
df_reg = df_clean.copy()
df_reg = pd.get_dummies(df_reg, columns=["category"], drop_first=True)

# Exclude id/name and target
exclude = ["product_id", "product_name", "profit", "cluster"]
feature_cols = [c for c in df_reg.columns if c not in exclude]
X = df_reg[feature_cols]
y = df_reg["profit"]

# Standardize numeric features in X (to help regression)
numeric_to_scale = [c for c in numeric_cols if c in X.columns and c != "profit"]
scaler_reg = StandardScaler()
if numeric_to_scale:
    X[numeric_to_scale] = scaler_reg.fit_transform(X[numeric_to_scale])

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=RANDOM_STATE)

# Model 1: Linear Regression
lr = LinearRegression()
lr.fit(X_train, y_train)
pred_lr = lr.predict(X_test)
mse_lr = mean_squared_error(y_test, pred_lr)
mae_lr = mean_absolute_error(y_test, pred_lr)

# Model 2: Polynomial Regression degree=2 (polynomial on main numeric features only)
poly_features = [f for f in ["price", "cost", "units_sold", "promotion_frequency", "shelf_level"] if f in X.columns]
poly = PolynomialFeatures(degree=2, include_bias=False)
X_poly_numeric = pd.DataFrame(poly.fit_transform(X[poly_features]),
                              columns=poly.get_feature_names_out(poly_features),
                              index=X.index)
# append any remaining (e.g., category dummies)
other_cols = [c for c in X.columns if c not in poly_features]
X_poly_full = pd.concat([X_poly_numeric, X[other_cols].reset_index(drop=True)], axis=1)

# split poly data consistent with previous split indices
X_poly_train = X_poly_full.loc[X_train.index]
X_poly_test = X_poly_full.loc[X_test.index]

pr = LinearRegression()
pr.fit(X_poly_train, y_train)
pred_pr = pr.predict(X_poly_test)
mse_pr = mean_squared_error(y_test, pred_pr)
mae_pr = mean_absolute_error(y_test, pred_pr)

# Save Actual vs Predicted plots
# Linear
plt.figure(figsize=(6,6))
plt.scatter(y_test, pred_lr, alpha=0.7)
mn, mx = min(y_test.min(), pred_lr.min()), max(y_test.max(), pred_lr.max())
plt.plot([mn, mx], [mn, mx], linestyle="--")
plt.title("Linear Regression: Actual vs Predicted (profit)")
plt.xlabel("Actual Profit")
plt.ylabel("Predicted Profit")
plt.grid(True)
plt.tight_layout()
plt.savefig(os.path.join(RESULTS_DIR, "actual_vs_pred_linear.png"))
plt.close()

# Polynomial
plt.figure(figsize=(6,6))
plt.scatter(y_test, pred_pr, alpha=0.7)
mn, mx = min(y_test.min(), pred_pr.min()), max(y_test.max(), pred_pr.max())
plt.plot([mn, mx], [mn, mx], linestyle="--")
plt.title("Polynomial (deg2) Regression: Actual vs Predicted (profit)")
plt.xlabel("Actual Profit")
plt.ylabel("Predicted Profit")
plt.grid(True)
plt.tight_layout()
plt.savefig(os.path.join(RESULTS_DIR, "actual_vs_pred_poly.png"))
plt.close()

# Regression summary
reg_summary = pd.DataFrame({
    "model": ["LinearRegression", "PolynomialDeg2"],
    "MSE": [mse_lr, mse_pr],
    "MAE": [mae_lr, mae_pr]
})

best_model_name = reg_summary.sort_values("MSE").iloc[0]["model"]
print("Regression summary:")
print(reg_summary.to_string(index=False))
print(f"Best model by MSE: {best_model_name}")

# -------------------------
# Build REPORT.pdf (text pages + images)
# -------------------------
with PdfPages(REPORT_PATH) as pdf:
    # Title page
    fig = plt.figure(figsize=(8.5, 11))
    fig.clf()
    fig.text(0.01, 0.96, "Machine Learning Assignment - Product Performance Analysis", fontsize=16, weight="bold")
    y = 0.9
    intro = ("This report documents preprocessing, a K-means clustering implementation from scratch,"
             " cluster analysis, and regression modeling (Linear and Polynomial degree 2)."
             " Figures and tables follow.")
    for line in textwrap.wrap(intro, 100):
        fig.text(0.01, y, line, fontsize=10)
        y -= 0.03
    fig.text(0.01, y-0.02, f"Dataset shape: {df.shape}", fontsize=9)
    plt.axis("off")
    pdf.savefig(fig)
    plt.close(fig)

    # Preprocessing summary page
    fig = plt.figure(figsize=(8.5, 11))
    fig.clf()
    fig.text(0.01, 0.98, "Data Preprocessing Summary", fontsize=12, weight="bold")
    y = 0.94
    mv_text = "Missing values (after imputation):\n" + df_clean.isnull().sum().to_string()
    for line in mv_text.splitlines():
        fig.text(0.01, y, line, fontsize=9)
        y -= 0.025
    y -= 0.02
    out_text = "Outlier handling: IQR winsorization (capped at 1.5*IQR bounds) for numeric features."
    for line in textwrap.wrap(out_text, 100):
        fig.text(0.01, y, line, fontsize=9)
        y -= 0.03
    pdf.savefig(fig)
    plt.close(fig)

    # Elbow image
    elbow_img = plt.imread(os.path.join(RESULTS_DIR, "elbow_curve.png"))
    fig = plt.figure(figsize=(8.5, 11))
    plt.imshow(elbow_img)
    plt.axis("off")
    pdf.savefig(fig)
    plt.close(fig)

    # Cluster scatter
    cluster_img = plt.imread(os.path.join(RESULTS_DIR, "cluster_price_units.png"))
    fig = plt.figure(figsize=(8.5, 11))
    plt.imshow(cluster_img)
    plt.axis("off")
    pdf.savefig(fig)
    plt.close(fig)

    # Cluster stats table
    fig = plt.figure(figsize=(8.5, 11))
    fig.clf()
    fig.text(0.01, 0.98, "Cluster Statistics", fontsize=12, weight="bold")
    y = 0.95
    for line in cluster_stats.to_string(index=False).splitlines():
        fig.text(0.01, y, line, fontsize=9)
        y -= 0.03
    pdf.savefig(fig)
    plt.close(fig)

    # Regression table
    fig = plt.figure(figsize=(8.5, 11))
    fig.clf()
    fig.text(0.01, 0.98, "Regression Model Comparison", fontsize=12, weight="bold")
    y = 0.95
    for line in reg_summary.to_string(index=False).splitlines():
        fig.text(0.01, y, line, fontsize=10)
        y -= 0.03
    pdf.savefig(fig)
    plt.close(fig)

    # Actual vs Predicted images
    avp_lin = plt.imread(os.path.join(RESULTS_DIR, "actual_vs_pred_linear.png"))
    fig = plt.figure(figsize=(8.5, 11))
    plt.imshow(avp_lin)
    plt.axis("off")
    pdf.savefig(fig)
    plt.close(fig)

    avp_poly = plt.imread(os.path.join(RESULTS_DIR, "actual_vs_pred_poly.png"))
    fig = plt.figure(figsize=(8.5, 11))
    plt.imshow(avp_poly)
    plt.axis("off")
    pdf.savefig(fig)
    plt.close(fig)

print(f"REPORT saved to {REPORT_PATH}")
print(f"Figures saved under {RESULTS_DIR}/")



Loaded product_sales.csv with shape (200, 9)
Recommended k (by silhouette): 2


/tmp/ipython-input-1589944154.py:247: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X[numeric_to_scale] = scaler_reg.fit_transform(X[numeric_to_scale])


Regression summary:
           model         MSE       MAE
LinearRegression 8161.764215 67.735024
  PolynomialDeg2 3069.763465 23.677905
Best model by MSE: PolynomialDeg2
REPORT saved to REPORT.pdf
Figures saved under results/


In [ ]:
from google.colab import files
uploaded = files.upload()


Saving product_sales.csv to product_sales.csv


In [ ]:
import os
print(os.listdir())

['.config', 'product_sales.csv', 'results', 'sample_data']
